In [ ]:
from oaklib import get_adapter

from malco.post_process.compute_mrr import mondo_adapter
# This takes in the raw results from Exomiser/Pheval and converts them into a TSV with 
# diff diagnosis and scores for each case:
# rank	score	disease_name	disease_identifier
# 1	0.8802	SCID, autosomal recessive, T-negative/B-positive type	OMIM:600802
# 2	0.868	Combined immunodeficiency with faciooculoskeletal anomalies	ORPHA:221139
# 2	0.868	Combined immunodeficiency with faciooculoskeletal anomalies	ORPHA:221139
# 4	0.8626	Severe combined immunodeficiency, X-linked	OMIM:300400

from malco.post_process.post_process_results_format import create_exomiser_standardised_results
from pathlib import Path

create_exomiser_standardised_results(
    results_dir=Path('/Volumes/files_2019/EXOMISER_PHENOTYPE_ONLY_2024_09_16/exomiser-14.0.1-2206/raw_results/'),
    output_dir=Path('/Users/jtr4v/PythonProject/malco/outputdir_all_2024_07_04/'),
    score_name='combined_score',
    sort_order='descending',
    variant_analysis=False,
    gene_analysis=False,
    disease_analysis=True,
    include_acmg=False,
)

Processing Exomiser Results:   6%|▌         | 323/5213 [22:53<8:01:51,  5.91s/it] 

In [3]:
from malco.post_process.mondo_score_utils import omim_mappings
from malco.post_process.compute_mrr import mondo_adapter

from warnings import warn

from oaklib.datamodels.vocabulary import IS_A
from oaklib.interfaces import MappingProviderInterface

from cachetools import cached, LRUCache
from cachetools.keys import hashkey

FULL_SCORE = 1
PARTIAL_SCORE = 0.5

# Now ground the results to MONDO and score as we did for GPT results. 

@cached(cache=LRUCache(maxsize=4096), info=True, key=lambda prediction, ground_truth, mondo_adapter: hashkey(prediction, ground_truth))
def score_exomiser_result(prediction: str,
                          ground_truth: str,
                          mondo_adapter,
                          orpha_curie_prefix: bool = True
                          ) -> float:
    """
    Score the prediction from Exomiser. The prediction is an OMIM or Orphanet ID.
    
    1) If the prediction is the same as the ground truth, return 1.0
    2) If the prediction is an OMIM or other ID that maps to a Mondo term whose descendent is the ground truth,  return PARTIAL_SCORE
    3) Otherwise, return 0.0
    
    This is similar essentially the same as score_grounded_result(), except we are
    mapping the predictions to Mondo then scoring 

    Args:
        prediction (str): The prediction.
        ground_truth (str): The ground truth.
        mondo_adapter: The mondo adapter.

    Returns:
        float: The score.
    """
    if not isinstance(mondo_adapter, MappingProviderInterface):
        raise ValueError("Adapter is not an MappingProviderInterface")
    
    if prediction == ground_truth:
        # predication is the correct OMIM
        return FULL_SCORE

    if ground_truth in omim_mappings(prediction, mondo_adapter):
        # prediction is an ID that directly maps to a correct OMIM
        return FULL_SCORE
    
    # Get mapping of the prediction to MONDO, then see if that Mondo or any of its descendants
    # map to the ground truth
    
    # Correct ORPHA curie prefix to Orphanet
    if orpha_curie_prefix:
        # Replace ^ORPHA: with Orphanet:
        if prediction.startswith("ORPHA:"):
            prediction = prediction.replace("ORPHA:", "Orphanet:")
            warn(f"Replaced ORPHA prefix with Orphanet for {prediction}")        
    
    these_mondos = [m['subject_id'] for m in list(mondo_adapter.sssom_mappings([prediction])) 
    if (m.predicate_id == "skos:exactMatch" or m.predicate_id == 'oio:hasDbXref') 
     and m.subject_source == 'MONDO']

    for mondo_descendant in mondo_adapter.descendants(these_mondos, predicates=[IS_A], reflexive=True):
        if ground_truth in omim_mappings(mondo_descendant, mondo_adapter):
            # prediction is a MONDO that maps to a correct OMIM via a descendant
            return PARTIAL_SCORE
    return 0.0

mondo_adapter = mondo_adapter()

assert(score_exomiser_result("OMIM:130050", "OMIM:130050", mondo_adapter) == FULL_SCORE) 
assert(score_exomiser_result("Orphanet:79255", "OMIM:230500", mondo_adapter) == PARTIAL_SCORE)
assert(score_exomiser_result("ORPHA:79255", "OMIM:230500", mondo_adapter) == PARTIAL_SCORE)
assert(score_exomiser_result("OMIM:130050", "OMIM:230500", mondo_adapter) == 0.0)  # Ehlers Danlos vascular type versus GM1 gangliosidosis type 1

/var/folders/vc/lfqgrrhn56d9yj5fbxbw6qr00000gt/T/ipykernel_53856/1157861670.py:60: UserWarning: Replaced ORPHA prefix with Orphanet for Orphanet:79255
  warn(f"Replaced ORPHA prefix with Orphanet for {prediction}")


In [4]:
# for each phenopacket file, find the corresponding exomiser results file
# make a dict of phenopacket_id -> exomiser_results_file

import os
import glob

# Define the directories
phenopacket_dir = "../supplemental_data/phenopackets/"
results_dir = "../outputdir_all_2024_07_04/pheval_disease_results/"

# Get all result files
result_files = glob.glob(os.path.join(results_dir, "*-pheval_disease_result.tsv"))

# Initialize the dictionary
results2phenopacket = {}

# Process each result file
for result_file in result_files:
    # Get the base name of the result file (without extension and suffix)
    base_name = os.path.basename(result_file).replace("-pheval_disease_result.tsv", "")
    
    # Construct the pattern for matching phenopacket files
    phenopacket_pattern = os.path.join(phenopacket_dir, f"{base_name}.json")
    
    # Find matching phenopacket files
    matching_phenopackets = glob.glob(phenopacket_pattern)
    
    # Check if there's exactly one matching phenopacket file
    if len(matching_phenopackets) == 1:
        results2phenopacket[os.path.basename(result_file)] = os.path.basename(matching_phenopackets[0])
    elif len(matching_phenopackets) == 0:
        raise ValueError(f"No matching phenopacket file found for {result_file}")
    else:
        raise ValueError(f"Multiple matching phenopacket files found for {result_file}")

print(f"Total mappings: {len(results2phenopacket)}")

Total mappings: 5212


In [6]:
import os
import glob
import json
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from oaklib import get_adapter
from warnings import warn

# number of lines to read from each result file after deduplication
num_lines_to_read = 16

# Define the directories
phenopacket_dir = "../supplemental_data/phenopackets/"
results_dir = "../outputdir_all_2024_07_04/pheval_disease_results/"
output_dir = "../outputdir_all_2024_07_04/pheval_disease_results_ADD_IS_CORRECT_COL/"

# Create output directory if it doesn't exist
Path(output_dir).mkdir(parents=True, exist_ok=True)

# Initialize the MONDO adapter for each file
mondo = get_adapter("sqlite:obo:mondo")

# Get all result files
result_files = glob.glob(os.path.join(results_dir, "*-pheval_disease_result.tsv"))

for result_file in tqdm(result_files, desc="Processing files"):
    try:
        # Get the base name of the result file (without extension and suffix)
        base_name = os.path.basename(result_file).replace("-pheval_disease_result.tsv", "")

        # Construct the path for the matching phenopacket file
        phenopacket_file = os.path.join(phenopacket_dir, f"{base_name}.json")

        # Check if the phenopacket file exists
        if not os.path.exists(phenopacket_file):
            warn(f"!!!!!!!!!!!!!!!!!!No matching phenopacket file found for {result_file}!!!!!!!!!!!!!!!!!")
            continue

        # Read the result file (reading all rows first)
        df = pd.read_csv(result_file, sep='\t')

        # Remove consecutive duplicates based on 'disease_identifier'
        df = df.loc[~df['disease_identifier'].duplicated(keep='first')]

        # Truncate the data after removing consecutive duplicates to keep only the first 'num_lines_to_read' rows
        df = df.head(num_lines_to_read)

        # Read the phenopacket file
        with open(phenopacket_file, 'r') as f:
            phenopacket_data = json.load(f)

        # Extract the correct disease ID
        correct_id = phenopacket_data['interpretations'][0]['diagnosis']['disease']['id']

        # Add the correct_ID column
        df['correct_ID'] = correct_id

        # Initialize a sub-progress bar for processing each row in the DataFrame
        with tqdm(total=len(df), desc=f"Processing rows in {base_name}") as row_progress_bar:
            # Calculate the score for each prediction
            df['grounded_score'] = df['disease_identifier'].apply(
                lambda x: score_exomiser_result(x, correct_id, mondo)
            )

            # Update progress bar after each row is processed
            for _ in df['grounded_score']:
                row_progress_bar.update(1)

        # Add the is_correct column (score of 1.0 means exact match)
        df['is_correct'] = df['grounded_score'] > 0  # 0.0 is False, 0.5 or 1.0 is True

        # Write the updated dataframe to the new directory
        output_file = os.path.join(output_dir, os.path.basename(result_file))
        df.to_csv(output_file, sep='\t', index=False)

    except Exception as e:
        print(f"Error processing {result_file}: {str(e)}")
        continue  # Skip to the next file

print("Processing completed successfully.")

Processing rows in PMID_27220909_proband: 100%|██████████| 16/16 [00:00<00:00, 28185.16it/s]

Processing rows in PMID_36446582_Ockeloen2015_P11: 100%|██████████| 16/16 [00:00<00:00, 46410.00it/s]

Processing rows in PMID_21683322_16: 100%|██████████| 16/16 [00:00<00:00, 30026.34it/s]

Processing rows in PMID_37962958_19: 100%|██████████| 16/16 [00:00<00:00, 43323.99it/s]

Processing rows in PMID_10580070_E_III-1:   0%|          | 0/16 [00:00<?, ?it/s]/var/folders/vc/lfqgrrhn56d9yj5fbxbw6qr00000gt/T/ipykernel_53856/1157861670.py:60: UserWarning: Replaced ORPHA prefix with Orphanet for Orphanet:130
  warn(f"Replaced ORPHA prefix with Orphanet for {prediction}")
/var/folders/vc/lfqgrrhn56d9yj5fbxbw6qr00000gt/T/ipykernel_53856/1157861670.py:60: UserWarning: Replaced ORPHA prefix with Orphanet for Orphanet:300751
  warn(f"Replaced ORPHA prefix with Orphanet for {prediction}")
/var/folders/vc/lfqgrrhn56d9yj5fbxbw6qr00000gt/T/ipykernel_53856/1157861670.py:60: UserWarning: Replaced ORPHA prefi

KeyboardInterrupt: 

In [ ]:
# import os
# import glob
# import pandas as pd
# import matplotlib.pyplot as plt
# from collections import defaultdict
# 
# # Define the directories
# dir1 = "../outputdir_all_2024_07_04/pheval_disease_results_WITH_IS_CORRECT/"
# dir2 = "../outputdir_all_2024_07_04/pheval_disease_results_WITH_IS_CORRECT_USING_MONDO/"
# 
# def get_correct_rank(file_path):
#     df = pd.read_csv(file_path, sep='\t')
#     correct_rows = df[df['is_correct'] == True]
#     return correct_rows['rank'].min() if not correct_rows.empty else float('inf')
# 
# results = defaultdict(lambda: {'dir1': float('inf'), 'dir2': float('inf')})
# 
# # Process files in both directories
# for directory in [dir1, dir2]:
#     for file_path in glob.glob(os.path.join(directory, "*.tsv")):
#         file_name = os.path.basename(file_path)
#         rank = get_correct_rank(file_path)
#         results[file_name]['dir1' if directory == dir1 else 'dir2'] = rank
# 
# # Analyze results
# dir1_better = 0
# dir2_better = 0
# equal = 0
# 
# for file_name, ranks in results.items():
#     if ranks['dir1'] < ranks['dir2']:
#         dir1_better += 1
#     elif ranks['dir2'] < ranks['dir1']:
#         dir2_better += 1
#     else:
#         equal += 1
# 
# print(f"Times dir1 (WITHOUT_MONDO) was better: {dir1_better}")
# print(f"Times dir2 (WITH_MONDO) was better: {dir2_better}")
# print(f"Times they were equal: {equal}")
# 
# # Create a bar plot
# labels = ['WITHOUT_MONDO better', 'WITH_MONDO better', 'Equal']
# values = [dir1_better, dir2_better, equal]
# 
# plt.figure(figsize=(10, 6))
# plt.bar(labels, values)
# plt.title('Comparison of Correct Diagnosis Ranks')
# plt.ylabel('Number of Cases')
# plt.savefig('rank_comparison.png')
# plt.close()
# 
# # Create a scatter plot
# plt.figure(figsize=(10, 6))
# x = [ranks['dir1'] for ranks in results.values()]
# y = [ranks['dir2'] for ranks in results.values()]
# plt.scatter(x, y, alpha=0.5)
# plt.plot([0, max(max(x), max(y))], [0, max(max(x), max(y))], 'r--')  # Diagonal line
# plt.xlabel('Rank in WITHOUT_MONDO')
# plt.ylabel('Rank in WITH_MONDO')
# plt.title('Comparison of Ranks between WITHOUT_MONDO and WITH_MONDO')
# plt.savefig('rank_scatter.png')
# plt.close()